In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1832,tt0215129,16000000,119754278,Road Trip,Breckin Meyer|Seann William Scott|Amy Smart|Pa...,Todd Phillips,The greatest college tradition of all.,Four friends take off on an 1800 mile road tri...,93,Comedy|Adventure,DreamWorks SKG|The Montecito Picture Company,5/19/2000,5.7,2000
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015
1047,tt1430615,40000000,24719215,Big Miracle,Drew Barrymore|John Krasinski|Kristen Bell|Vin...,Ken Kwapis,A family of whales trapped beneath the ice. On...,"Based on an inspiring true story, a small-town...",107,Adventure|Drama|Romance,Universal Pictures|Anonymous Content|Working T...,2/2/2012,6.5,2012
277,tt1182345,5000000,9760104,Moon,Sam Rockwell|Kevin Spacey|Dominique McElligott...,Duncan Jones,The last place you'd ever expect to find yours...,With only three weeks left in his three year c...,97,Science Fiction|Drama,Liberty Films UK|Lunar Industries|Xingu Films|...,6/12/2009,7.4,2009
1563,tt0338013,20000000,72258126,Eternal Sunshine of the Spotless Mind,Jim Carrey|Kate Winslet|Elijah Wood|Mark Ruffa...,Michel Gondry,I already forget how I used to feel about you.,"Joel Barish, heartbroken that his girlfriend u...",108,Science Fiction|Drama|Romance,Anonymous Content|This Is That Productions|Foc...,3/19/2004,7.8,2004


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [33]:
answers = {} # создадим словарь для ответов

data['profit'] = data['revenue'] - data['budget'] 
#Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. 
#(прибыль = сборы - бюджет) 
#в нашем датасете это будет (profit = revenue - budget)

data['release_date'] = pd.to_datetime(data['release_date'])
#приводим время к удобному формату

# добавляем столбец месяцев (№23)
months = {'1': 'Январь', '2': 'Февраль', '3': 'Март', '4': 'Апрель',
          '5': 'Май', '6': 'Июнь', '7': 'Июль', '8': 'Август', '9': 'Сентябрь',
          '10': 'Октябрь', '11': 'Ноябрь', '12': 'Декабрь'}
data['month'] = data['release_date'].dt.month
data['month'] = data['month'].apply(lambda x: months[str(x)])

# функция для средней (№25)
def length_calculation(data_mini):
    companies = pd.Series(
        data_mini['production_companies'].str.cat(sep='|').split('|'))
    companies = companies.value_counts(ascending=False)
    for company in companies.index:
        sum_len = data_mini['length'][data_mini['production_companies'].map(
            lambda x: True if company in x.split('|') else False)].sum()
        companies[company] = sum_len / companies[company]

    return companies.sort_values(ascending=False)

# 1. У какого фильма из списка самый большой бюджет?

In [35]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
#+
# если ответили верно, можете добавить комментарий со значком "+"

In [9]:
data.sort_values(by=['budget'], ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000,Май


2-й вариант

In [11]:
data.loc[data['budget'].idxmax()][:4]

imdb_id                                             tt1298650
budget                                              380000000
revenue                                            1021683000
original_title    Pirates of the Caribbean: On Stranger Tides
Name: 723, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [36]:
answers['2'] = '2. Gods and Generals (tt0279111)'

In [16]:
data[data['runtime'] == data['runtime'].max()][['original_title', 'runtime']]

,original_title,runtime
1157,Gods and Generals,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [37]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [27]:
data[data['runtime'] == data['runtime'].min()][['original_title', 'runtime']]

,original_title,runtime
768,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [38]:
answers['4'] = '2. 110'

In [29]:
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [39]:
answers['5'] = '1. 107'

In [31]:
round(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [41]:
answers['6'] = '5. Avatar (tt0499549)'

In [42]:
data[data['profit'] == data['profit'].max()][['original_title']]

,original_title
239,Avatar


# 7. Какой фильм самый убыточный? 

In [43]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [44]:
data[data['profit'] == data['profit'].min()][['original_title']]

,original_title
1245,The Lone Ranger


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [45]:
answers['8'] = '1. 1478'

In [46]:
len(data[data['profit'] > 0])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [47]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [48]:
data[['revenue', 'original_title']][data['release_year']==2008].sort_values('revenue', ascending=False)[:2]

,revenue,original_title
599,1001921825,The Dark Knight
603,786636033,Indiana Jones and the Kingdom of the Crystal S...


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [49]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [50]:
data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,Июль


# 11. Какого жанра фильмов больше всего?

In [68]:
answers['11'] = '3. Drama'

In [69]:
pd.Series(data['genres'].str.cat(sep='|').split('|')).value_counts()[:3]

Drama       782
Comedy      683
Thriller    596
dtype: int64

2-й вариант

In [70]:
Counter(data['genres'].str.cat(sep='|').split('|')).most_common(3)

[('Drama', 782), ('Comedy', 683), ('Thriller', 596)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [71]:
answers['12'] = '1. Drama'

In [72]:
data_profit = data[data['profit'] > 0]

In [73]:
pd.Series((data_profit.genres.str.split('|').sum())).value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [78]:
answers['13'] = '5. Peter Jackson'

In [77]:
data_revenue = data.groupby(['director']).sum()
data_revenue[data_revenue.profit == data_revenue.profit.max()]

,budget,revenue,runtime,vote_average,release_year,profit
director,,,,,,
Peter Jackson,1288000000,6490593685,1355,58.1,16059,5202593685


2-й вариант

In [79]:
directors = set(data.director.str.split('|').sum())
pd.Series({x: data[data.director.str.contains(x)].revenue.sum() for x in directors}).sort_values(ascending = False).head(3)

Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [104]:
answers['14'] = '3. Robert Rodriguez'

In [82]:
action_genres = data[data.genres.str.contains("Action", na=False)]
director_count = pd.Series(
    action_genres['director'].str.cat(sep='|').split('|'))
some_director = director_count.value_counts(ascending=False)
display(some_director.sort_values(ascending=False))

Robert Rodriguez       9
Paul W.S. Anderson     7
Michael Bay            7
Ridley Scott           6
Antoine Fuqua          6
                      ..
Sean McNamara          1
Alexander Witt         1
Ernest R. Dickerson    1
Feng Xiaogang          1
Mike Newell            1
Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [83]:
answers['15'] = '3. Chris Hemsworth'

In [84]:
data_12 = data.query('release_year == 2012')
cast = pd.Series(data_12['cast'].str.cat(sep='|').split('|'))
cast = cast.value_counts(ascending=False)
for actor in cast.index:
    cast[actor] = data_12['revenue'][data_12['cast'].map(
        lambda x: True if actor in x else False)].sum()
cast = cast.sort_values(ascending=False)
display(cast.index[0])

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [85]:
answers['16'] = '3. Matt Damon'

In [86]:
pd.Series(data[data.budget > data.budget.mean()].cast.str.split('|').sum()).value_counts().index[0]

'Matt Damon'

2-й вариант

In [88]:
data_1 = data[data.budget > data.budget.mean()]
Counter(data_1.cast.str.split('|').sum()).most_common(1)

[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [91]:
answers['17'] = '2. Action'

In [93]:
Counter(data[data.cast.str.contains('Nicolas Cage')].genres.str.split('|').sum()).most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [95]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [96]:
data[data['production_companies'].str.contains('Paramount Pictures', na=False)].sort_values(by='profit').head(1)['original_title'].values[0]

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [99]:
answers['19'] = '5. 2015'

In [100]:
best_profit = pd.pivot_table(data, values='revenue', index='release_year', aggfunc='sum')
best_profit.revenue.sort_values(ascending=False).head(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [101]:
answers['20'] = '1. 2014'

In [102]:
data[data.production_companies.str.contains('Warner Bros')].groupby('release_year')['profit'].sum().idxmax()

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [106]:
answers['21'] = '4. Сентябрь'

In [107]:
data.groupby(data['release_date'].dt.strftime('%B'))['imdb_id'].count().sort_values(ascending = False)

release_date
September    227
December     190
October      186
August       161
March        156
April        149
June         147
November     146
July         142
May          140
February     135
January      110
Name: imdb_id, dtype: int64

2-й вариант

In [108]:
data_month = data.groupby(data['month'])['original_title'].count()
data_month = data_month.sort_values(ascending=False)
display(data_month)
print(data_month.index[0])

month
Сентябрь    227
Декабрь     190
Октябрь     186
Август      161
Март        156
Апрель      149
Июнь        147
Ноябрь      146
Июль        142
Май         140
Февраль     135
Январь      110
Name: original_title, dtype: int64

Сентябрь


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [109]:
answers['22'] = '2. 450'

In [110]:
print(len(data[(data.release_date.dt.month >=6) & (data.release_date.dt.month <=8)]))

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [111]:
answers['23'] = '5. Peter Jackson'

In [113]:
data_winter = data[data['month'].isin(['Декабрь', 'Январь', 'Февраль'])]
director_count = pd.Series(data_winter['director'].str.cat(sep='|').split('|'))
some_director = director_count.value_counts(ascending=False)
print(some_director.index[0])

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [115]:
answers['24'] = '5. Four By Two Productions'

In [116]:
data['len_title'] = data.original_title.apply(lambda x: len(x))

In [117]:
data.groupby(['production_companies'])['len_title'].mean().sort_values(ascending=False)[:3]

production_companies
Twentieth Century Fox Film Corporation|Four By Two Productions        83.0
Walt Disney|Walden Media                                              62.0
Walt Disney Pictures|21 Laps Entertainment|Jim Henson Company, The    59.0
Name: len_title, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [121]:
answers['25'] = '3. Midnight Picture Show'

In [119]:
data_mini = data.loc[:, ('overview', 'production_companies')]
data_mini['length'] = [len(x.split(' ')) for x in data_mini['overview']]
# используем функции из предобработки
companies = length_calculation(data_mini)
companies.index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [122]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [123]:
data.loc[data['vote_average']>data.quantile(0.99)['vote_average']]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

2-й вариант

In [124]:
top_films = ['Inside Out', 'The Dark Knight', '12 Years a Slave',
             'BloodRayne', 'The Adventures of Rocky & Bullwinkle',
             'Batman Begins', 'The Lord of the Rings: The Return of the King',
             'Upside Down', '300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1']
data_average = data.sort_values(
    ['vote_average', 'original_title'], ascending=False)  
last_procent = data_average.head(len(data) // 100)  
last_procent.query('original_title in @top_films')[
    ['original_title', 'vote_average']]

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
1081,The Lord of the Rings: The Return of the King,7.9
1191,12 Years a Slave,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [125]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [128]:
top_couples_1 = Counter()
for i in range(len(data)):
    temp_list = data.iloc[i]['cast'].split('|')
    for actor1 in temp_list:
        new_list = temp_list.copy()
        new_list.remove(actor1)
        for actor2 in new_list:
            top_couples_1[(actor1, actor2)] += 1

top_couples_1.most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

In [132]:
top_couples_2 = pd.Series([['Ben Stiller', 'Owen Wilson'], ['Paul Walker', 'Vin Diesel'],
                         ['Helena Bonham Carter', 'Johnny Depp'], [
                             'Adam Sandler', 'Kevin James'],
                         ['Daniel Radcliffe', 'Rupert Grint']])
couples = (data['cast'].str.split('|'))
for actor in top_couples_2.index:
    filter_actor = [x for x in couples if (
        top_couples_2[actor][0] in x) and (top_couples_2[actor][1] in x)]
    top_couples_2[actor] = len(filter_actor)
print(top_couples_2)


0    6
1    5
2    6
3    5
4    8
dtype: object


# Submission

In [133]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [134]:
# и убедиться что ни чего не пропустил)
len(answers)

27